In [1]:
from PIL import Image
import zipfile
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
import zipfile
f=zipfile.ZipFile('./cats2.zip','r')
for file in f.namelist():
    f.extract(file) #解压到的位置，./表示当前目录(与此.ipynb文件同一个目录)
f.close()

In [ ]:
def read_directory(directory_name):
    array_of_img=[]
    for filename in os.listdir(r"./"+directory_name):
        img = cv2.imread(directory_name + "/" + filename,0)
        array_of_img.append(img)
    return array_of_img

In [2]:
dataset=pd.read_csv('./train_list .csv')
dataset=np.array(dataset)

In [3]:
list_name=[]
list_label2=[]
for  i in range(2160):
    if i<=1799:
        list_name.append(dataset[i][0][0:-2])
        list_label2.append(int(dataset[i][0][-1]))
    if i>1799:
        list_name.append(dataset[i][0][0:-3])
        list_label2.append(int(dataset[i][0][-2:]))
            

In [ ]:
arr_im=[]
for i in list_name:
    img=cv2.imread(i,0)
    arr_im.append(img)

In [ ]:
all_pic=arr_im

In [ ]:
len(all_pic)

In [ ]:
pic=[]
list_label=[]

In [ ]:
for i in range(2160):
    try:
        im1=Image.fromarray(all_pic[i])
        pic.append(np.array(im1.resize((400,400))))
        list_label.append(list_label2[i])
    except:
        AttributeError

In [ ]:
len(list_label)

In [ ]:
pic1=[]
list_label1=[]
for i in range(179):
    for j in range(12):
        pic1.append(pic[179*j+i])
        list_label1.append(list_label[179*j+i])

In [ ]:
len(list_label1)

In [ ]:
data_train=[]
data_train_label=[]
data_test=[]
data_test_label=[]

In [ ]:
data_train=pic1[0:1800]
data_train_label=list_label1[0:1800]
data_test=pic1[1800:]
data_test_label=list_label1[1800:]

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# 用于ResNet18和34的残差块，用的是2个3x3的卷积
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        # 经过处理后的x要与x的维度相同(尺寸和深度)
        # 如果不相同，需要添加卷积+BN来变换为同一维度
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


# 用于ResNet50,101和152的残差块，用的是1x1+3x3+1x1的卷积
class Bottleneck(nn.Module):
    # 前面1x1和3x3卷积的filter个数相等，最后1x1卷积是其expansion倍
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes,
                               kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=12):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(1, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(4608*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3])

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])


def test():
    net = ResNet18()
    y = net(torch.randn(1,3,32,32))
    print(y.size())

# test()

In [ ]:
len(data_train)

In [ ]:
a_plus=[]
b_plus=[]

In [ ]:
for i in range(50):
    a_plus.append([])
    b_plus.append([])
    for j in range(36):
        a_plus[i].append(data_train[i*36+j])
        b_plus[i].append(data_train_label[i*36+j])
        

In [ ]:
for i in range(50):
    a_plus[i]=torch.tensor(a_plus[i]).reshape(len(a_plus[i]),1,400,400)
    b_plus[i]=torch.tensor(b_plus[i]).reshape(len(b_plus[i]),)

In [ ]:
a_plus[0].shape

In [ ]:
model=ResNet50() 
criterion=nn.CrossEntropyLoss() 
optimizer=optim.SGD(model.parameters(),lr=0.005,momentum=0.9)

In [ ]:
def train(model,epoch,criterion,optimizer,data_train,data_train_label):
    model.train()
    while (epoch>0):
        for data,target in zip(data_train,data_train_label):
            data,target=Variable(data).float(),Variable(target).long()
            output=model(data)
            optimizer.zero_grad()
            loss=criterion(output,target)
            loss.backward()
            optimizer.step()
        epoch-=1

In [ ]:
train(model,5,criterion,optimizer,a_plus,b_plus)

In [ ]:
c_plus=[]
d_plus=[]

In [ ]:
c=torch.tensor(data_test[0:]).reshape(348,1,100,100)

In [ ]:
d=torch.tensor(data_test_label[0:]).reshape(348,)

In [ ]:
for i in range(6):
    c_plus.append(torch.tensor(data_test[i*50:(i+1)*50]).reshape(50,1,100,100))
    d_plus.append(torch.tensor(data_test_label[i*50:(i+1)*50]).reshape(50,))

In [ ]:
c_plus.append(torch.tensor(data_test[300:]).reshape(48,1,100,100))
d_plus.append(torch.tensor(data_test_label[300:]).reshape(48,))

In [ ]:
d_plus[0]

In [ ]:
def test(model,epoch,criterion,data_test,data_test_label):
    model.eval()
    test_loss=0
    correct=0
    #for data,target in zip(data_test,data_test_label):
    data,target=Variable(data_test).float(),Variable(data_test_label).long()
    output=model(data)
    test_loss+=criterion(output,target).item()
    pred=output.data.max(1)[1]
    print(pred)
    correct+=pred.eq(target).cpu().sum()
    test_loss/=50
    acc=correct/50
    return (acc,test_loss),output.data.max(1)[1]

In [ ]:
test(model,1,criterion,c_plus[0],d_plus[0])